In [94]:
# Important libaries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [95]:
# plot Accurcy graph for model tuning?
plot_knn_accuracy = False
plot_logreg_accuracy = False

In [96]:
# Read TSV data
columns = ["ID", "label_6", "statement", "subject", "speaker", "speaker_job", "state", "party", "pof_count", "false_count", "barelytrue_count", "halftrue_count", "mostlytrue_count", "context"]
train_data = pd.read_table("/kaggle/input/LIARbyYang/train.tsv", header = None, names = columns)
v_data = pd.read_table("/kaggle/input/LIARbyYang/valid.tsv", header = None, names = columns)
test_data = pd.read_table("/kaggle/input/LIARbyYang/test.tsv", header = None, names = columns)


Here we do not want to merge the three tables provided. Next, check the data:

In [97]:
train_data.head(10)

,ID,label_6,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


In [98]:
train_data.shape

(10240, 14)

# Relabel

We want to merge the labels from 6-ways to 2 and 3-ways, while keeping the original 6-ways labels for the benchmark.

In [99]:
merger_2 = { 'pants-fire' : 0,
           'false' : 0,
           'barely-true': 0,
           'half-true': 0,
           'mostly-true': 1,
           'true': 1}
merger_3 = { 'pants-fire' : 0,
           'false' : 0,
           'barely-true': 1,
           'half-true': 1,
           'mostly-true': 2,
           'true': 2}
merger_6 = { 'pants-fire' : 0,
           'false' : 1,
           'barely-true': 2,
           'half-true': 3,
           'mostly-true': 4,
           'true': 5}

# training set
train_data['label_2'] = train_data["label_6"].map(merger_2)
train_data['label_3'] = train_data["label_6"].map(merger_3)
train_data["label_6"].replace(merger_6, inplace = True)

# cross validation set
v_data['label_2'] = v_data["label_6"].map(merger_2)
v_data['label_3'] = v_data["label_6"].map(merger_3)
v_data["label_6"].replace(merger_6, inplace = True)

# test set
test_data['label_2'] = test_data["label_6"].map(merger_2)
test_data['label_3'] = test_data["label_6"].map(merger_3)
test_data["label_6"].replace(merger_6, inplace = True)

# check
train_data.head(10)

,ID,label_6,statement,subject,speaker,speaker_job,state,party,pof_count,false_count,barelytrue_count,halftrue_count,mostlytrue_count,context,label_2,label_3
0,2635.json,1,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0,0
1,10540.json,3,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,0,1
2,324.json,4,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,1,2
3,1123.json,1,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0,0
4,9028.json,3,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,0,1
5,12465.json,5,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,1,2
6,2342.json,2,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.,0,1
7,153.json,3,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa.",0,1
8,5602.json,3,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website,0,1
9,9741.json,4,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video,1,2


# Bag of Words Process

In [100]:
# tokenization processes


from nltk.corpus import stopwords
import string
from nltk.tokenize import WhitespaceTokenizer as w_tokenizer
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer("english")
stop_words = set(stopwords.words('english'))

# remove stopwords
def remove_stopwords(list_of_tokens):
    new_list_of_tokens = [w for w in list_of_tokens if not w.lower() in stop_words]
    return new_list_of_tokens

# remove punctutations and tokens with less than two in length
def remove_punctuations_shortwords(list_of_tokens):
    translation = str.maketrans('', '', string.punctuation)
    new_list_of_tokens = [tokens.translate(translation) for tokens in list_of_tokens if len(tokens.translate(translation))>=3]
    return new_list_of_tokens

def tokenize_and_stem(Pandas_Series):
    Pandas_Series = Pandas_Series.replace('\d+', '', regex=True) # remove numbers
    Pandas_Series = Pandas_Series.apply(nltk.word_tokenize) # tokenize statement
    Pandas_Series = Pandas_Series.apply(remove_stopwords) # remove stopwords
    Pandas_Series = Pandas_Series.apply(remove_punctuations_shortwords) # remove punctutations and tokens with less than two in length
    Pandas_Series = Pandas_Series.apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
    return Pandas_Series

In [101]:
train_data['stemmed_statement_token'] = tokenize_and_stem(train_data['statement']) # Process statement strings

See sentence length distribution:

In [ ]:
sns.histplot(train_data['stemmed_statement_token'].str.len())

<Axes: xlabel='stemmed_statement_token', ylabel='Count'>

One can see that there are some outlier sentences that exceed 30 tokens. Our approach is not to get rid of their records, but to exclude them from the CountVecterizer fitting. To do that, we keep the index of instances with statements having too much tokens.

In [ ]:
statement_index_tofit = train_data['stemmed_statement_token'].str.len() < 30
train_data[-statement_index_tofit] # instances with very long statements

See the most common (and potentially useless) words for the dataset, check for redundant words:

In [ ]:
# bag of words for the whole dataset
def countwords(x):
    word2count = {}
    for words in x:
        for word in words:
            if word not in word2count.keys():
                word2count[word] = 1
            else:
                word2count[word] += 1
    # sort the word2count
    sorted_word2count = sorted(word2count.items(), key=lambda x:x[1], reverse = True)
    return sorted_word2count
            
words_in_statements = countwords(train_data["stemmed_statement_token"])
words_in_statements[0:20]

Consider the n-grams

In [ ]:
# n-grams
def build_bigrams(token):
    if len(token) >= 1:
        x = list(nltk.ngrams(token, 2))
    else:
        x = []
    return x

def build_trigrams(token):
    if len(token) >= 2:
        x = list(nltk.ngrams(token, 3))
    else:
        x = []
    return x

train_data["stemmed_statement_bigrams"]=train_data["stemmed_statement_token"].apply(build_bigrams)
train_data["stemmed_statement_trigrams"]=train_data["stemmed_statement_token"].apply(build_trigrams)

Apply CountVectorizer, ignore the first few rendundant words and words that appear only once.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
v_statement = CountVectorizer(analyzer=lambda x: x, max_df = 1.0 ,min_df = 2) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
v_statement.fit(train_data['stemmed_statement_bigrams'][statement_index_tofit])
vectors = v_statement.transform(train_data['stemmed_statement_bigrams'])

def new_features(X, string, vectorizer):
    new_features = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out()) # Get new features from countvectorizer
    if type(new_features.columns[0]) == tuple:
        new_features.rename(columns='_'.join, inplace=True)
    new_features.columns = string + '_' + new_features.columns # add prefix to the column names
    return new_features

statement_vector = new_features(vectors, 'st', v_statement)

In [ ]:
statement_vector.columns

In [ ]:
sns.histplot(np.sum(statement_vector, axis = 0))

Merge new features

In [ ]:
train_data2 = pd.concat([train_data,statement_vector],axis=1)

In [ ]:
train_data2.head()

Do the same for context. But there are 102 null present, drop them first.

In [ ]:
sum(train_data2['context'].isnull())

Suppose null value for the context is caused by unknown source, replace them with "unknown".

In [ ]:
# fill null context
train_data2["context"].fillna(value = "unknown", inplace=True)

In [ ]:
train_data2['stemmed_context_token'] = tokenize_and_stem(train_data2['context']) # Process context strings

Check redundant:

In [ ]:
words_in_context = countwords(train_data2["stemmed_context_token"])
print(words_in_context[0:20])

In [ ]:
len(words_in_context)

Apply vectorizer:

In [ ]:
v_context = CountVectorizer(analyzer=lambda x: x, max_df = 1800 ,min_df = 50) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
vectors = v_context.fit_transform(train_data2['stemmed_context_token'])
context_vector = new_features(vectors, 'ct', v_context)

In [ ]:
context_vector.columns

In [ ]:
sns.histplot(np.sum(context_vector, axis = 0))

In [ ]:
train_data3 = pd.concat([train_data2,context_vector],axis=1)
train_data3.shape

Do the same for subject. Now, the tokens are separated by commas in this case. Thus, change them to spaces before tokenizing them.

In [ ]:
sum(train_data3['subject'].isnull()) #check number of nulls

In [ ]:
train_data3[train_data3['subject'].isnull()]

There are two nulls to get rid of. Replace them with "unknown".

In [ ]:
# fill null subject
train_data3["subject"].fillna(value = "unknown", inplace=True)

Apply defined tokenizer:

In [ ]:
train_data3['subject'].replace(',',' ',regex=True,inplace=True)  # turn commas to blank spaces
train_data3['stemmed_subject_token'] = tokenize_and_stem(train_data3['subject']) # Process subject strings

Check redundant:

In [ ]:
words_in_subject = countwords(train_data3["stemmed_subject_token"])
print(words_in_subject[0:20])

Apply CountVectorizer:

In [ ]:
v_subject = CountVectorizer(analyzer=lambda x: x, max_df = 1.0 ,min_df = 50) # need analyzer=lambda x: x to disable the analyzer, since the input is pandas series of lists.
vectors = v_subject.fit_transform(train_data3['stemmed_subject_token'])
subject_vector = new_features(vectors, 'sj', v_subject)

In [ ]:
subject_vector.columns

In [ ]:
sns.histplot(np.sum(subject_vector, axis = 0))

In [ ]:
train_data4 = pd.concat([train_data3,subject_vector],axis=1)

Drop original columns and get the final DataFrame:

In [ ]:
train_data4.drop(columns=['statement', 'stemmed_statement_token','context', 'stemmed_context_token','subject', 'stemmed_subject_token'], inplace=True)

In [ ]:
train_data4.shape

In [ ]:
train_data4.head(10)

# Training Data Cleaning and Encoding

In [ ]:
print(np.sum(train_data4.isnull(),axis = 0)) 

First, we ignore the speaker's name and focus on their past speech counts. Even if the counts themselves may not determine if they will speak the truth in the future or not. But in terms of our work, they can serve as vectors determining who is which person. Then, for speaker jobs and their states, we deem that they are less likely to contribute to the model comparing to other atributes. Thus, let's verify if they are really the features to get rid of.

In [ ]:
print('Speaker\'s job and states absolute correlation to 6-ways label:', np.abs(train_data4[['speaker_job','state']].corrwith(train_data4['label_6'],method= 'kendall')))
print('Speaker\'s jobs and states absolute correlation to 3-ways label:', np.abs(train_data4[['speaker_job','state']].corrwith(train_data4['label_3'],method= 'kendall')))
print('Speaker\'s jobs and states absolute correlation to 2-ways label:', np.abs(train_data4[['speaker_job','state']].corrwith(train_data4['label_2'],method= 'kendall')))

Check if there is some individual feartures that are useful.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(handle_unknown = 'ignore') #ignore tells the encoder to ignore new categories by encoding them with 0's
vectors = onehot.fit_transform(np.array(train_data4[['speaker_job','state']]))
speaker_vectors = new_features(vectors, 'speaker', onehot)

In [ ]:
print('Top 5 speaker\'s job and state absolute correlation to 6-ways label:', np.abs(speaker_vectors.corrwith(train_data4['label_6'],method= 'pearson').nlargest(n=5)))
print('Top 5 speaker\'s job and state absolute correlation to 3-ways label:', np.abs(speaker_vectors.corrwith(train_data4['label_3'],method= 'pearson').nlargest(n=5)))
print('Top 5 speaker\'s job and state absolute correlation to 2-ways label:', np.abs(speaker_vectors.corrwith(train_data4['label_2'],method= 'pearson').nlargest(n=5)))

One can see that the correlations to the target class do not exceed 0.034. Also, there is no individual useful feature; thus, confirm our assumption that they are irrelavant. Proceed to drop them together with statement ID and speaker's name.

In [ ]:
# drop unnescessary columns
train_data4.drop(columns = ['ID','speaker','speaker_job','state'],inplace = True)

Next, there are some null for the counts and party:

In [ ]:
np.sum(train_data4[['pof_count','false_count','barelytrue_count','halftrue_count','mostlytrue_count']].isnull(),axis = 0)

In [ ]:
np.sum(train_data['party'].isnull(),axis = 0)

Assume that these are caused by unknown speaker, simply set them to 0.

In [ ]:
train_data4['pof_count'].fillna(0, inplace = True)
train_data4['false_count'].fillna(0, inplace = True)
train_data4['barelytrue_count'].fillna(0, inplace = True)
train_data4['halftrue_count'].fillna(0, inplace = True)
train_data4['mostlytrue_count'].fillna(0, inplace = True)

and assume "unknown" for the null in party.

In [ ]:
train_data4['party'].fillna("unknown", inplace = True)

Check the party feature correlation to the target classes:

In [ ]:
print('Speaker\'s party absolute correlation to 6-ways label:', np.abs(train_data4['party'].corr(train_data4['label_6'],method= 'kendall')))
print('Speaker\'s party absolute correlation to 3-ways label:', np.abs(train_data4['party'].corr(train_data4['label_3'],method= 'kendall')))
print('Speaker\'s party absolute correlation to 2-ways label:', np.abs(train_data4['party'].corr(train_data4['label_2'],method= 'kendall')))

Turns out our assumption about the party is wrong. The feature is not as useful as we thought. Nonetheless, let's investigate a little deeper if we can extract any useful feature. First, One-Hot encode party via scikit-learn module:

In [ ]:
ohe = OneHotEncoder(handle_unknown = 'ignore') #ignore tells the encoder to ignore new categories by encoding them with 0's
vectors = ohe.fit_transform(np.array(train_data4['party']).reshape(-1, 1))
party_vector = new_features(vectors, 'party', ohe)

In [ ]:
print('Top 5 speaker\'s party absolute correlation to 6-ways label:', np.abs(party_vector.corrwith(train_data4['label_6'],method= 'pearson').nlargest(n=5)))
print('Top 5 speaker\'s party absolute correlation to 3-ways label:', np.abs(party_vector.corrwith(train_data4['label_3'],method= 'pearson').nlargest(n=5)))
print('Top 5 speaker\'s party absolute correlation to 2-ways label:', np.abs(party_vector.corrwith(train_data4['label_2'],method= 'pearson').nlargest(n=5)))

The Democrat stands out from the other top 5 parties. Therefore, we decide to keep this in our model.

In [ ]:
train_data4 = pd.concat([train_data4,party_vector['party_x0_democrat']],axis=1)
train_data4.drop(columns=['party'], inplace=True)

In [ ]:
train_data4.head(10)

# Training, Valdiation and Test Sets Preparation

Apply BoW, vectorizer, and one-hot encoding to v and test set without word counts threshold and cut-offs. Then, mask the columns with ones from the training set, such that the columns that do not appear in the training set are dropped. For columns that do not exist in the v and test set, add them and set their values to zero.

In [ ]:
# fill nulls
v_data["subject"].fillna(value = "unknown", inplace=True)
v_data["context"].fillna(value = "unknown", inplace=True)
v_data['party'].fillna("unknown", inplace = True)
test_data["subject"].fillna(value = "unknown", inplace=True)
test_data["context"].fillna(value = "unknown", inplace=True)
test_data['party'].fillna("unknown", inplace = True)

v_data['pof_count'].fillna(0, inplace = True)
v_data['false_count'].fillna(0, inplace = True)
v_data['barelytrue_count'].fillna(0, inplace = True)
v_data['halftrue_count'].fillna(0, inplace = True)
v_data['mostlytrue_count'].fillna(0, inplace = True)

test_data['pof_count'].fillna(0, inplace = True)
test_data['false_count'].fillna(0, inplace = True)
test_data['barelytrue_count'].fillna(0, inplace = True)
test_data['halftrue_count'].fillna(0, inplace = True)
test_data['mostlytrue_count'].fillna(0, inplace = True)

In [ ]:
# tokenize statement
v_data["stemmed_statement_token"] = tokenize_and_stem(v_data['statement'])
test_data["stemmed_statement_token"] = tokenize_and_stem(test_data['statement'])

# tokenize context
v_data["stemmed_context_token"] = tokenize_and_stem(v_data['context'])
test_data["stemmed_context_token"] = tokenize_and_stem(test_data['context'])

# turn commas to blank spaces for subjects
v_data['subject'].replace(',',' ',regex=True,inplace=True) 
test_data['subject'].replace(',',' ',regex=True,inplace=True)  

# tokenize subject
v_data["stemmed_subject_token"] = tokenize_and_stem(v_data['subject'])
test_data["stemmed_subject_token"] = tokenize_and_stem(test_data['subject'])

In [ ]:
# build n-grams

v_data["stemmed_statement_bigrams"] = v_data["stemmed_statement_token"].apply(build_bigrams)
v_data["stemmed_statement_trigrams"] = v_data["stemmed_statement_token"].apply(build_trigrams)

test_data["stemmed_statement_bigrams"] = test_data["stemmed_statement_token"].apply(build_bigrams)
test_data["stemmed_statement_trigrams"] = test_data["stemmed_statement_token"].apply(build_trigrams)

Apply the countvectorizer based on what we have done for the training set via transform.

In [ ]:
def transformtovec(Pandas_Series, prefixstring, vectorizer):
    vectors = vectorizer.transform(Pandas_Series)
    vecdataframe = new_features(vectors, prefixstring, vectorizer)
    return vecdataframe

In [ ]:
# Statement
# get new DataFrame
v_statement_vectors = transformtovec(v_data["stemmed_statement_bigrams"], 'st', v_statement)
test_statement_vectors = transformtovec(test_data["stemmed_statement_bigrams"], 'st', v_statement)

# Context
# get new DataFrame
v_context_vectors = transformtovec(v_data["stemmed_context_token"], 'ct', v_context)
test_context_vectors = transformtovec(test_data["stemmed_context_token"], 'ct', v_context)

# Context
# get new DataFrame
v_subject_vectors = transformtovec(v_data["stemmed_subject_token"], 'sj', v_subject)
test_subject_vectors = transformtovec(test_data["stemmed_subject_token"], 'sj', v_subject)

In [ ]:
# merge the data in the same order as the training set

# cross validation set
v_data = pd.concat([v_data,v_statement_vectors],axis=1)
v_data = pd.concat([v_data,v_context_vectors],axis=1)
v_data = pd.concat([v_data,v_subject_vectors],axis=1)

# test set
test_data = pd.concat([test_data,test_statement_vectors],axis=1)
test_data = pd.concat([test_data,test_context_vectors],axis=1)
test_data = pd.concat([test_data,test_subject_vectors],axis=1)

# drop original and unnecessary columns
v_data.drop(columns=['ID','statement', 'stemmed_statement_token','context', 'stemmed_context_token','subject', 'stemmed_subject_token','speaker','speaker_job','state'], inplace=True)
test_data.drop(columns=['ID','statement', 'stemmed_statement_token','context', 'stemmed_context_token','subject', 'stemmed_subject_token','speaker','speaker_job','state'], inplace=True)

Get the Democrat One-Hot Feature:

In [ ]:
# validation
v_vectors = ohe.transform(np.array(v_data['party']).reshape(-1, 1))
v_party_vector = new_features(v_vectors, 'party', ohe)
v_data = pd.concat([v_data,v_party_vector['party_x0_democrat']],axis=1)
v_data.drop(columns=['party'], inplace=True)

# test
test_vectors = ohe.transform(np.array(test_data['party']).reshape(-1, 1))
test_party_vector = new_features(test_vectors, 'party', ohe)
test_data = pd.concat([test_data,test_party_vector['party_x0_democrat']],axis=1)
test_data.drop(columns=['party'], inplace=True)

Lastly, properly identify the independent and dependent variables.

In [ ]:
# features
X_train = train_data4.drop(columns = ['label_2','label_3','label_6'])
X_v = v_data.drop(columns = ['label_2','label_3','label_6'])
X_test = test_data.drop(columns = ['label_2','label_3','label_6'])

# targets
y_train = train_data4[['label_2','label_3','label_6']]
y_v = v_data[['label_2','label_3','label_6']]
y_test = test_data[['label_2','label_3','label_6']]

In [ ]:
label = 2 # label to metric: 0,1, 2 for 2, 3 and 6 ways respectively

# Naive Bayes


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train.iloc[:,label])
y_pred = gnb.predict(X_v)

In [ ]:
from sklearn import metrics
print(metrics.accuracy_score(y_v['label_6'], y_pred))

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

k_range = list(range(1, 250,10))
scores = []
scores2 = []
score_cv = []
for i in k_range:
    knn = KNeighborsClassifier(n_neighbors=i,weights = 'distance')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_v)
    y_pred2 = knn.predict(X_test)
    cv_scores = cross_val_score(knn, X_train, y_train.iloc[:,label], cv=5)
    scores.append(metrics.accuracy_score(y_v.iloc[:,label], y_pred[:,label]))
    score_cv.append(cv_scores.mean())
    scores2.append(metrics.accuracy_score(y_test.iloc[:,label], y_pred2[:,label]))

plt.plot(k_range, scores, label = 'val')
plt.plot(k_range, score_cv, label = 'cv 5 fold')
plt.plot(k_range, scores2, label = 'test')
plt.legend()
plt.show()
print(np.transpose(np.array([k_range,np.around(scores2,7)*100])))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# n = 31
knn = KNeighborsClassifier(n_neighbors=91,weights = 'distance')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
cm = confusion_matrix(y_test.iloc[:,label],y_pred[:,label], labels=knn.classes_[label])
print(metrics.accuracy_score(y_test.iloc[:,label], y_pred[:,label]))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=knn.classes_[label])
disp.plot()
plt.show()

In [ ]:
if plot_knn_accuracy:
    k_range = list(range(1, 100,5))
    score_train = []
    score_v = []
    score_test = []
    for i in k_range:
        knn = KNeighborsClassifier(n_neighbors=i, weights = 'distance',algorithm = 'brute',metric = 'manhattan')
        knn.fit(X_train, y_train)
        y_pred_train = knn.predict(X_train)
        v_scores = cross_val_score(knn, X_train, y_train, v=5)
        y_pred_test = knn.predict(X_test)
        score_train.append(metrics.accuracy_score(y_train, y_pred_train))
        score_v.append(v_scores.mean())
        score_test.append(metrics.accuracy_score(y_test, y_pred_test))
    
    np.array(score_train).tofile('KNN_score_train.csv', sep = ',')   
    np.array(score_v).tofile('KNN_score_v.csv', sep = ',')   
    np.array(score_test).tofile('KNN_score_test.csv', sep = ',') 

In [ ]:
if plot_knn_accuracy:
    score_train = pd.read_csv("/kaggle/working/KNN_score_train.csv", header = None)
    score_v = pd.read_csv("/kaggle/working/KNN_score_v.csv", header = None)
    score_test = pd.read_csv("/kaggle/working/KNN_score_test.csv", header = None)

    #k_range = list(range(1, 60,5))

    plt.plot(k_range, score_train.T ,label = 'train')
    plt.plot(k_range, score_v.T,label = 'v 5 fold')
    plt.plot(k_range, score_test.T,label = 'test')
    plt.legend()
    plt.xlabel('k')
    plt.ylabel('Accuracy score')
    plt.yscale('log')
    plt.show()

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

if plot_logreg_accuracy:
    order_range = np.arange(-4,-2,0.2)
    c_range = 10.0**order_range
    scores = []
    for c in c_range:
        logreg = LogisticRegression(random_state=0,C=c,max_iter = 500)
        logreg.fit(X_train, y_train)
        y_pred = logreg.predict(X_v)
        scores.append(metrics.accuracy_score(y_v, y_pred))

In [ ]:
if plot_logreg_accuracy:
    plt.plot(order_range, scores)
    plt.show()
    print(np.transpose(np.array([order_range,np.around(scores,7)*100])))

# Test

In [ ]:
train_data4.head(10).filter(regex='statement').columns

In [ ]:
train_data4.head(10)

In [ ]:
train_data4.head(10).filter(regex='statement')

In [ ]:
#sns.pairplot(train_data4[['pof_count','false_count','barelytrue_count','halftrue_count','mostlytrue_count','context_ad','label_2']],hue = 'label_2')

In [ ]:
from copy import deepcopy
df_test = deepcopy(train_data)
df_test['subject'] = train_data['subject'].mask(train_data.groupby('subject')['subject'].transform('size').lt(60), 'Others')
df_test.drop(df_test[df_test['subject'] ==  'Others'].index, inplace = True)

Show unique value for "context"

Show all rows that contain:

In [ ]:
np.sum(test_data.filter(regex='count').isnull(),axis = 0)